In [ ]:
import sys # required for relative imports in jupyter lab
sys.path.insert(0, '../') 

from cosmosis.model import tv_model, ResBam
from cosmosis.learning import Learn, Selector
from cosmosis.dataset import LoadImage, AsTensor, Transpose
import cosmosis.dataset as dataset

from dataset import Ranzcr
from learning import RSelector

from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms

from IPython.display import display


In [ ]:
ds_params={'embed': [],
            'image_dir': './data/ranzcr/train',
            'target_csv': './data/ranzcr/train.csv',
            'target_type': 'ETT',
            'transform': transforms.Compose([LoadImage(),
                                             transforms.Resize(224),
                                             ]),
            'target_transform': None}

ds = Ranzcr(**ds_params)
ids = dataset.ImageDatasetStats()

for i in ds.ds_idx:
    ids(ds[i][0])
    
print(ids.stats.mean) #123.2/255=.483
print(ids.stats.stddev) #59.4/255=.233

In [ ]:
ds_params={'train_params': {'embed': [],
                            'image_dir': './data/ranzcr/train',
                            'target_csv': './data/ranzcr/train.csv',
                            'target_type': 'ETT',
                            'transform': transforms.Compose([LoadImage(),
                                                             transforms.Resize(1500),
                                                             transforms.RandomRotation(10),
                                                             transforms.CenterCrop(1200),
                                                             transforms.Resize(600),
                                                             transforms.RandomCrop(448),
                                                             transforms.ToTensor(),
                                                             transforms.Normalize(.483,.233),
                                                             ]),
                            'target_transform': transforms.Compose([
                                                       dataset.AsTensor(),
                                                       dataset.Squeeze()])},
           'val_params': {'embed': [],
                          'image_dir': './data/ranzcr/train',
                          'target_csv': './data/ranzcr/train.csv',
                          'target_type': 'ETT',
                          'transform': transforms.Compose([LoadImage(),
                                                           transforms.Resize(1500),
                                                           transforms.CenterCrop(1200),
                                                           transforms.Resize(600),
                                                           transforms.CenterCrop(448),
                                                           transforms.ToTensor(),
                                                           transforms.Normalize(.483,.233),
                                                           ]),
                          'target_transform': transforms.Compose([
                                                       dataset.AsTensor(),
                                                       dataset.Squeeze()])},
           'test_params': {'embed': [],
                           'image_dir': './data/ranzcr/train',
                           'target_csv': './data/ranzcr/train.csv',
                           'target_type': 'ETT',
                           'transform': transforms.Compose([LoadImage(),
                                                            transforms.Resize(1500),
                                                            transforms.CenterCrop(1200),
                                                            transforms.Resize(600),
                                                            transforms.CenterCrop(448),
                                                            transforms.ToTensor(),
                                                            transforms.Normalize(.483,.233),
                                                            ]),
                          'target_transform': transforms.Compose([
                                                       dataset.AsTensor(),
                                                       dataset.Squeeze()])}}

model_params = {'n_classes': 4,
                'residuals': True,
                'in_channels': 1}

metrics_params = {'report_interval': 10, 
                  'sk_metric_name': 'roc_auc_score', 
                  'sk_params': {'average': 'macro',
                                'multi_class': 'ovr'}}
    
opt_params = {'lr': 0.01}

crit_params = {'reduction': 'sum'}

#RSelector balances classes (target_type) of the Ranzcr ds
sample_params = {'set_seed': False,
                 'splits': (.7,.15),
                 'target_type': 'ETT'}

sched_params = {'factor': .5,
                'patience': 4,
                'cooldown': 2}

learn = Learn([Ranzcr,Ranzcr,Ranzcr], 
              ResBam, 
              RSelector, 
              Optimizer=Adam, 
              Scheduler=ReduceLROnPlateau, 
              Criterion=CrossEntropyLoss, 
              model_params=model_params, ds_params=ds_params, sample_params=sample_params,
              opt_params=opt_params, sched_params=sched_params, crit_params=crit_params,
              metrics_params=metrics_params,
              adapt=False, load_model=False, load_embed=False, save_model=False,
              batch_size=16, epochs=20)